**MOVIE RECOMMENDATION ENGINE**

   Bu projede GroupLens araştırma grubunun bir araya getirdiği veri seti kullanılmıştır. MovieLens veri setinde kullanıcı bilgileri, film bilgileri, kullanıcıların filmleri derecelendirdiği bilgiler yer almaktadır.Ayrıca MovieLens kaydolabileceğiniz, film inceleyip katkıda bulunabileceğiniz, film önerileri alabileceğiniz bir web sitesidir.

   Pandas python programlama dili için yüksek performanslı, kullanımı kolay veri yapıları ve veri analiz araçları sağlayan açık kaynaklı bir BSD lisanslı kütüphanedir. Csv ve text dosyalarını açmaya ve  içerisinde bulunan verileri okuyarak istenen sonuca  kolayca ulaşmak için kullanılmaktadır.
   Numpy kütüphanesi bilimsel hesaplama  işlemleri kolaylaştırmak için yazılmış olan bir python kütüphanesidir. 
   Aşağıda pandas ve numpy import edilmiştir.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import csv
import surprise
from surprise import Reader, Dataset
from surprise import accuracy
from surprise import SVD
import random
import matplotlib.pyplot as plt # data visualization library
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS #used to generate world cloud
import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

Daha sonra "pandas read_csv()" kullanarak dataset yüklenir. İlk olarak movies.csv dosyası okundu. Ekrana shape ile (satır, sütun) sayıları yazdırıldı.

In [ ]:
df1= pd.read_csv('../input/movies.csv')
df1.shape

Şimdi verileri görmek için head() fonksiyonu çağırıldı. Görmek istenilen veri kadar değer girilebilir.

In [ ]:
df1.head(10)

Aynı şekilde filmlere verilen ratings değerlerini okumak için pandas read_csv() kullanarak dosya okundu. Ve ekrana (satır,sütun) sayıları yazdırıldı.

In [ ]:
df2= pd.read_csv('../input/ratings.csv')
df2.shape

In [ ]:
df2.head(10)

Son olarak links.csv dosyasını da okuduk.

In [ ]:
df3= pd.read_csv('../input/links.csv')
df3.shape
df3.head()

In [ ]:
title= pd.read_csv('../input/movies.csv')
title.head(10)

Aşağıda merge fonksiyonuyla, movie.csv ve ratings dosyalarının ortak sütunları "movieId" ye göre birleştirildi ve yeni bir tablo oluşturuldu.

Farklı kullanıcıların aynı filme verdiği ratings değerleri görüntülendi.

**Her sütunun neyi temsil ettiğine bakalım:**

userId - Filmi derecelendiren kullanıcının kimliği.

movieId - Filmin kimliği.

rating - Kullanıcının filmi verdiği derecelendirme, 1 ile 5 arasında.

timestamp - Filmin derecelendirildiği saat.

title - Filmin başlığı.

In [ ]:
df2 = pd.merge(df2,title, on='movieId')
df2.head(20)

Üzerinde çalıştığımız veriyi daha iyi anlamak için describe() ya da info() komutunu kullanarak verilerin kısa bir açıklamasını görebiliriz.

Örneğin aşağıda veri kümesinin 27278 kaydının olduğunu görüyoruz.

In [ ]:
df1.describe()

Veri kümesinin ratings.csv dosyasında 0'dan 1048575'e kadar numaralandırılmış girdi olduğunun bilgisini alıyoruz. Her bir sütunda null olmayan girdi sayısı ve sütunun veri tipi gibi bilgileri elde edebiliriz.

In [ ]:
df2.info()

describe() metodu sayısal verilere sahip olan sütunların max, min , std…gibi istatiksel değerlerini döndürür. 

In [ ]:
df2.describe()

Her film için ortalama derecelendirmeye ve derecelendirme sayısına sahip bir dataframe oluşturuldu. Daha sonra filmler arasındaki korelasyonu hesaplamak için bu derecelendirmeler kullanılacak. Korelasyon, iki veya daha fazla değişkenin birlikte dalgalanma derecesini gösteren istatistiksel bir ölçüdür. Korelasyon katsayısı yüksek olan filmler birbirine en çok benzeyen filmlerdir. Bu projede **Pearson korelasyon katsayısı** kullanıldı. Bu sayı -1 ile 1 arasında olacaktır. 1, pozitif bir korelasyonu gösterirken -1, negatif bir korelasyonu gösterir. 0, doğrusal bir korelasyon olmadığını gösterir. Bu nedenle, sıfır korelasyonlu filmler hiç aynı değildir. Bu veri çerçevesini oluşturmak için **pandas groupby** işlevi kullanıldı. Veri kümesini title sütununa göre gruplandırıp, her film için ortalama puanı almak için ortalaması hesaplandı.

In [ ]:
ratings = pd.DataFrame(df2.groupby('title')['rating'].mean())
ratings.head(10)



Her film için derecelendirme sayısını görmek isteriz. Biz bunu bir **number_of_ratings** sütunu oluşturarak yaparız. Bu, bir filmin ortalama derecesi ile filmin aldığı derecelendirme sayısı arasındaki ilişkiyi görebilmemiz için önemlidir. 5 yıldızlı bir filmin sadece bir kişi tarafından derecelendirilmiş olması olası bir durumdur.Bu nedenle, filmin 5 yıldızlı bir filmi olduğunu sınıflandırmak istatistiksel olarak yanlıştır. Bu nedenle, tavsiye sistemini kurarken minimum derecelendirme sayısı için bir eşik ayarlamamız gerekir. Bu yeni sütunu oluşturmak için **pandas groupby()** yardımcı programını kullanıyoruz, başlık sütununa göre gruplandırıyoruz ve daha sonra her bir filmin aldığı derecelendirme sayısını hesaplamak için **count()** işlevini kullanıyoruz, daha sonra head () işlevini kullanarak yeni dataframe' i görüntülüyoruz.

In [ ]:
ratings['number_of_ratings'] = df2.groupby('title')['rating'].count()
ratings.head(10)


Şimdi derecelendirmelerin dağılımını görselleştirmek için pandas çizim işlevini kullanarak bir Histogram çizelim.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
ratings['rating'].hist(bins=50)

Filmlerin çoğunun 2,5 ile 4 arasında derecelendirildiğini görebiliriz. Daha sonra number_of_ratings sütununu benzer şekilde görüntüleyelim.

In [ ]:
ratings['number_of_ratings'].hist(bins=60)

Yukarıdaki histogramdan, çoğu filmin çok az derecelendirmesi olduğu açıktır, en çok derecelendirmeye sahip filmler en meşhur olanlardır.


Şimdi bir filmin reytingi ile reyting sayısı arasındaki ilişkiyi kontrol edelim: Bunu seaborn kullanarak, bir dağılım grafiği çizerek yapıyoruz. Seaborn, jointplot () işlevini kullanarak bunu yapmamızı sağlıyor.

In [ ]:
import seaborn as sns
sns.jointplot(x='rating', y='number_of_ratings', data=ratings)

Diyagramdan, bir filmin ortalama derecelendirmesi ile derecelendirme sayısı arasında pozitif bir ilişki olduğunu görebiliriz. Grafik, bir filmin derecelendirmesi arttıkça aldığı ortalama derecelendirmesinin yükseldiğini gösterir. Buna, özellikle film başına derecelendirme sayısı eşiğini seçerken dikkat edilmesi gerekir.

Veri setimizi, film başlıklarını sütun olarak, userId değerini dizin olarak  ve rating'i value olarak konumlandıran bir matrise dönüştürmemiz gerekir. Bunu yaparak, film başlıkları olan sütunlarla ve kullanıcı kimlikleri olan satırlarla bir dataframe elde edeceğiz. Her sütun, bir filmin tüm kullanıcılar tarafından derecelendirmesini temsil eder. Kullanıcının belirli bir filmi derecelendirmediği durumlarda rating değeri NAN olarak görünür. Bu matrisi, tek bir filmin derecelendirmeleri ile matristeki geri kalan filmlerin arasındaki korelasyonu hesaplamak için kullanacağız. Film matrisini oluşturmak için **pandas pivot_table** yardımcı programını kullanıyoruz.

In [ ]:
movie_matrix = df2.pivot_table(index='userId', columns='title', values='rating')
movie_matrix.head(50)

**Pandalas sort_values** ​​yardımcı programını kullanıyoruz. Pandas sort_values ​​() işlevi, veritabanını artan veya azalan sütun sırasına göre sıralar.  

Ve filmleri en çok puan alanına göre düzenlemek için false olarak ayarlıyoruz. İlk 10’u görüntüleme işlevi için head() metodunu çağırıyoruz.

In [ ]:
ratings.sort_values('number_of_ratings', ascending=False).head(10)

Bir kullanıcının Air Force One'ı (1997) ve Contact'ı (1997) izlediğini varsayalım. Bu izleyici geçmişine dayanarak bu kullanıcıya filmler önermek istiyoruz. Amaç kullanıcıya Contact (1997) ve Air Force One'a (1997) benzeyen filmlere bakmaktır. Bunu, bu iki filmin reytingleri ile kalanların reytingleri arasındaki korelasyonu hesaplayarak başarabiliriz. İlk adım, movie_matrix'den  bu filmlerin derecelendirmelerini içeren bir dataframe oluşturmaktır.

In [ ]:
AFO_user_rating = movie_matrix['Air Force One (1997)']
contact_user_rating = movie_matrix['Contact (1997)']

Şimdi userId ve iki filme verdikleri derecelendirmeleri gösteren dataframe oluşturuldu. Aşağıda görüntülendi.

In [ ]:
AFO_user_rating.head()
contact_user_rating.head(20)

İki dataframe arasındaki korelasyonu hesaplamak için **pandas corwith ** metodunu kullanıyoruz, Corwith ile iki dataframe nesnesinin satır veya sütunlarının çift korelasyonunu hesaplar. Her bir filmin derecesi ile Air Force One filminin reytingleri arasındaki korelasyonu elde etmek için bu işlevi kullanıldı.


In [ ]:
similar_to_air_force_one=movie_matrix.corrwith(AFO_user_rating)

Air Force One filmi ile 'night Mother (1986) arasındaki ilişkinin 0.667291 olduğunu görebiliriz. Bu iki film arasında çok güçlü bir benzerlik olduğunu gösterir.

In [ ]:
similar_to_air_force_one.head(20)

In [ ]:
similar_to_contact = movie_matrix.corrwith(contact_user_rating)

Contact (1997) filmi reytingleri ile kalan film reytingleri arasındaki korelasyonu hesaplayalım. Prosedür yukarıda kullanılanla aynıdır.

In [ ]:
similar_to_contact.head(20)

Hesaplamadan All the Marbles(California Dolls, The) (1981)  filmi ile çok güçlü bir korelasyon olduğu görülmektedir.

Daha önce farkedildiği gibi, matrisimizin tüm filmleri tüm kullanıcılar tarafından derecelendirilmediği için çok fazla eksik değeri vardı. Bu nedenle, bu boş değerleri düşürüyoruz ve sonuçların daha çekici görünmesi için korelasyon sonuçlarını dataframe' e dönüştürüyoruz.

In [ ]:
corr_contact = pd.DataFrame(similar_to_contact, columns=['Correlation'])
corr_contact.dropna(inplace=True)
corr_contact.head()
corr_AFO = pd.DataFrame(similar_to_air_force_one, columns=['correlation'])
corr_AFO.dropna(inplace=True)
corr_AFO.head()

Yukarıdaki bu iki dataframe bize sırasıyla Contact (1997) ve Air Force One (1997) filmlerine en çok benzeyen filmleri göstermektedir. Ancak, bazı filmlerin çok az derecelendirmeye sahip olması ve bir ya da iki kişinin onlara 5 yıldız derecesi vermesi nedeniyle önerilmesinin son derece zor olması gibi bir problem vardır. Bunu, derecelendirme sayısı için bir eşik belirleyerek düzeltebiliriz. Daha önce yapılan histogramda, derecelendirme sayısında 100'den keskin bir düşüş gördük. Bu nedenle bunu eşik olarak ayarlayacağız, ancak bu, uygun bir seçenek elde edene kadar oynayabileceğiniz bir sayıdır. Bunu yapmak için, iki veri çerçevesine ratings dataframe'indeki number_of_ratings sütununu eklememiz gerekir.

In [ ]:
corr_AFO = corr_AFO.join(ratings['number_of_ratings'])
corr_contact = corr_contact.join(ratings['number_of_ratings'])
corr_AFO .head()
corr_contact.head()

Şimdi Air Force One'a (1997) en çok benzeyen filmleri, en az 100 yorumu olan filmlerle sınırlandırarak elde edeceğiz. Onları **correlation **sütununa göre sıralar ve ilk 10'u görüntüleriz.

In [ ]:
corr_AFO[corr_AFO['number_of_ratings'] > 100].sort_values(by='correlation', ascending=False).head(10)


Air Force One'ın (1997) kendisiyle mükemmel bir ilişkisinin olduğu görülmektedir. Air Force One (1997) filmine en çok benzeyen bir sonraki film Girl with the Dragon Tattoo, The (2011) ile arasında 0.775995 korelasyon var. Görüldüğü gibi inceleme sayısının eşiğini değiştirerek, önceki yöntemden farklı sonuçlar alıyoruz. Reyting sayısını sınırlandırmak bize daha iyi sonuçlar veriyor ve yukarıdaki filmleri Air Force One'ı (1997) izleyen birine güvenle tavsiye edebiliriz.


Şimdi Contact (1997) filmi için aynı şeyi yapalım ve ona en çok uyan filmleri görelim.

In [ ]:
corr_contact[corr_contact['number_of_ratings'] > 100].sort_values(by='Correlation', ascending=False).head(10)


Bir kez daha farklı sonuçlar alıyoruz. Contact (1997) filmine en çok benzeyen film, 0.515617 korelasyon katsayısına sahip 146 derecelendirme sayısı ile Thomas Crown Affair, The (1968). Birisi Contact (1997) adlı kişiyi beğense, yukarıdaki filmleri onlara tavsiye edebiliriz.

**DÜZELTME**


**Single Value Decomposition / Tekil değer ayrışımı	(SVD) (SVD) **

> Doğrusal cebirde, tekil değer ayrışımı (SVD) gerçek veya karmaşık bir matrisin çarpanlara ayrılmasıdır. 

Kullanıcıya verilen öğelerin derecelendirilmesinin ne kadar iyi olduğunu görebilmek için kullanılan yaygın bir metrik olan **RMSE** yani **Kök Ortalama Kare Hatası** hesaplandı. RMSE ne kadar düşük olursa performans o kadar iyidir.  RMSE değerinin sıfır olması modelin hiç hata yapmadığı anlamına gelir.


SVD'yi uygulamak için **Suprise **kütüphanesini kullanacağız. Suprise kütüphanesiyle algoritmaları kıyaslayacağız. Ayrıca pandas dataframe'den veri setini yüklemek için bir **Reader** nesnesine ihtiyacımız var. Aşağıda oluşturulmuştur.

In [ ]:
from surprise import Reader, Dataset, SVD, evaluate
reader = Reader()
ratings = pd.read_csv('../input/ratings.csv')
ratings.head(10)

In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [ ]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])


Ortalama 0.83 ortalama bir Kök Ortalama Kare Hatası(RMSE) alıyoruz ki bu da yeterince iyi. Şimdi veri setimiz hakkında eğitim verelim ve tahminlere varalım.

"Kullandığımız modelin ne kadar sağlıklı, kullanılabir olduğunu test etmek için veri setimizi trainin ve test olarak iki parçaya ayırırız. Trainin veri seti, modelimiz tarafından kullanılır ve bir bağıntı kurar. Modelimizin ne kadar sağlıklı sonuç ürettiğini test etmek için Test setini kullanırız. Böylece modelin kullanılıp kullanılamayacağına karar vermiş oluruz."

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Kullanıcı kimliği 1 olan kullanıcıyı seçip, derecelendirme yaptığı filmleri listeleyelim.

In [132]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826


In [135]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=4.074002037801348, details={'was_impossible': False})

Örneğin userId=1 kullanıcısının, 303 Id numaralı film için tahmini bir derecelendirme alırız.(est)